In [ ]:
 import pandas as pd

In [ ]:
# import pandas as pd
# df = pd.read_csv ("/content/canonical_jpop_dataset_final.csv")
# df2 = df[['artist', 'title', 'lyrics', 'summary', 'tags_normalized']]
# df2.to_csv("lyrics_tags_inspectation_kw_jk.csv")

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install selenium tqdm pandas python-Levenshtein thefuzz requests
!apt-get update
!apt-get install -y chromium-browser xvfb

import time
import random
import pandas as pd
import os
import re
from urllib.parse import urljoin, quote
from tqdm.auto import tqdm
import requests  # 이미지 다운로드를 위해 추가

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from thefuzz import fuzz

# ==============================================================================
# 2. Colab 환경을 위한 셀레니움 및 기본 설정
# ==============================================================================
INPUT_FILE = '/content/canonical_jpop_dataset_final.csv'
OUTPUT_FILE = 'jpop_lyrics_collection.csv'
FAILED_FILE = 'failed_list.txt'
IMAGE_DIR = 'album_covers'  # 앨범 커버 저장 폴더
SIMILARITY_THRESHOLD = 65

# [추가] 앨범 커버 저장 폴더 생성
os.makedirs(IMAGE_DIR, exist_ok=True)

def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f'--user-data-dir=/tmp/chrome_user_data_{random.randint(10000, 99999)}')
    chrome_options.add_argument("--disable-gpu")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    return driver

def normalize_string(text):
    text = re.sub(r'\(.*?\)|\[.*?\]', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s\u3040-\u30ff\u3131-\uD79D\u4e00-\u9fff]', '', text)
    return text.lower().strip()

def clean_search_query(artist, title):
    cleaned_title = re.sub(r'\s*feat\..*$', '', title, flags=re.IGNORECASE)
    cleaned_title = re.sub(r'\s*prod\..*$', '', cleaned_title, flags=re.IGNORECASE)
    cleaned_artist = artist.replace('.', '')
    return cleaned_artist, cleaned_title

# [추가] 파일명으로 사용할 수 없는 문자 제거 함수
def sanitize_filename(name):
    """파일명에 포함될 수 없는 문자를 제거하거나 대체합니다."""
    return re.sub(r'[\\/*?:"<>|]', "", name)

# [추가] 이미지 다운로드 함수
def download_image(url, path):
    """주어진 URL의 이미지를 지정된 경로에 저장합니다."""
    try:
        response = requests.get(url, stream=True, timeout=15)
        response.raise_for_status()  # HTTP 오류가 발생하면 예외를 발생시킴
        with open(path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"  ✅ 이미지 저장 성공: {os.path.basename(path)}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"  - 이미지 다운로드 실패: {e}")
        return False

# ==============================================================================
# 3. 사이트별 가사/앨범커버 수집 함수 (벅스 전용)
# ==============================================================================
def scrape_from_bugs(driver, artist, title):
    """벅스 '곡' 탭에서 검색하여 가사와 앨범 커버 URL을 수집하는 함수"""
    search_query = f"{artist} - {title}"
    base_url = "https://music.bugs.co.kr"
    search_url = f"{base_url}/search/track?q={quote(search_query)}"

    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 10)
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'trackList')]")))
        except TimeoutException:
            return None # 검색 결과 없음

        search_results = driver.find_elements(By.XPATH, "//table[contains(@class, 'trackList')]/tbody/tr")
        if not search_results:
            return None

        for result in search_results:
            try:
                found_title = result.find_element(By.CSS_SELECTOR, 'p.title a').text
                found_artist = result.find_element(By.CSS_SELECTOR, 'p.artist a').text

                title_score = fuzz.partial_ratio(normalize_string(title), normalize_string(found_title))
                artist_score = fuzz.partial_ratio(normalize_string(artist), normalize_string(found_artist))

                if title_score >= SIMILARITY_THRESHOLD and artist_score >= SIMILARITY_THRESHOLD:
                    print(f"  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 {title_score}%, 아티스트 {artist_score}%)")
                    track_info_link_element = result.find_element(By.CSS_SELECTOR, 'a.trackInfo')
                    relative_link = track_info_link_element.get_attribute('href')

                    # [개선] 벅스 고유 트랙 ID 추출 (파일명에 사용)
                    bugs_track_id = re.search(r'/track/(\d+)', relative_link).group(1)

                    song_link_full = urljoin(base_url, relative_link)
                    driver.get(song_link_full)
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.basicInfo')))

                    lyrics, album_cover_url = None, None

                    # 1. 가사 추출
                    try:
                        lyrics_element = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]//xmp')
                        lyrics = lyrics_element.get_attribute("innerHTML").strip()
                        print("  🎶 벅스 가사 수집 성공!")
                    except NoSuchElementException:
                        lyrics = "[가사 없음]" # 가사가 없는 경우도 있으므로 처리
                        print("  ⚠️ 벅스에 가사가 제공되지 않습니다.")

                    # 2. 앨범 커버 URL 추출
                    try:
                        cover_img_element = driver.find_element(By.CSS_SELECTOR, 'div.basicInfo div.photos img')
                        low_res_url = cover_img_element.get_attribute('src')
                        # 고화질 URL로 변경 (Bugs URL 구조 활용)
                        album_cover_url = low_res_url.replace('/images/200/', '/images/1000/')
                        print(f"  📸 앨범 커버 URL 확보!")
                    except NoSuchElementException:
                        album_cover_url = None # 앨범 커버가 없는 경우 처리
                        print("  ⚠️ 앨범 커버를 찾을 수 없습니다.")

                    return {'source': 'Bugs', 'lyrics': lyrics, 'translation':'', 'album_cover_url': album_cover_url, 'bugs_track_id': bugs_track_id}
            except (NoSuchElementException, TimeoutException):
                continue # 현재 항목에서 오류 발생 시 다음 검색 결과로

    except Exception as e:
        print(f"  - 벅스 검색/수집 중 예외 발생: {e}")

    return None

In [ ]:
# ==============================================================================
# 4. 메인 실행 로직
# ==============================================================================
print("1단계: 데이터 사전 준비 시작...")
if not os.path.exists(INPUT_FILE):
    print(f"❌ 오류: 원본 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 파일을 업로드해주세요.")
else:
    # 원본 목록 로드
    source_df = pd.read_csv(INPUT_FILE)
    unique_songs_df = source_df[['artist', 'title']].drop_duplicates().reset_index(drop=True)

    # [개선] 기존 수집 결과 로드
    try:
        results_df = pd.read_csv(OUTPUT_FILE)
        # 앨범 커버 URL 컬럼이 없으면 추가
        if 'album_cover_url' not in results_df.columns:
            results_df['album_cover_url'] = None
    except (FileNotFoundError, pd.errors.EmptyDataError):
        results_df = pd.DataFrame(columns=['artist', 'title', 'source', 'lyrics', 'translation', 'album_cover_url', 'bugs_track_id'])

    # [개선] 수집 대상을 더 정확하게 결정
    # - 아직 수집되지 않았거나, 수집되었지만 앨범 커버 URL이 없는 곡들만 대상
    merged_df = pd.merge(unique_songs_df, results_df, on=['artist', 'title'], how='left', suffixes=('', '_res'))
    to_scrape_df = merged_df[merged_df['album_cover_url'].isnull()].reset_index(drop=True)

    if to_scrape_df.empty:
        print("\n🎉 모든 곡의 가사와 앨범 커버 수집이 이미 완료되었습니다!")
    else:
        print(f"\n2단계: 총 {len(unique_songs_df)}곡 중 {len(to_scrape_df)}곡에 대한 수집/업데이트를 시작합니다...")
        if os.path.exists(FAILED_FILE):
            os.remove(FAILED_FILE)

        # [개선] 모든 결과를 담을 리스트
        all_results = results_df.to_dict('records')

        for index, row in tqdm(to_scrape_df.iterrows(), total=len(to_scrape_df), desc="J-POP 가사/커버 수집 중"):
            original_artist, original_title = row['artist'], row['title']
            cleaned_artist, cleaned_title = clean_search_query(original_artist, original_title)

            print(f"\n🔍 수집 시도 ({index + 1}/{len(to_scrape_df)}): {original_artist} - {original_title}")
            driver, scraped_data = None, None
            try:
                driver = setup_driver()
                scraped_data = scrape_from_bugs(driver, cleaned_artist, cleaned_title)

                if scraped_data and scraped_data.get('bugs_track_id'):
                    # 이미지 파일명 생성 및 다운로드
                    if scraped_data.get('album_cover_url'):
                        sanitized_artist = sanitize_filename(original_artist)
                        sanitized_title = sanitize_filename(original_title)
                        image_filename = f"{scraped_data['bugs_track_id']}_{sanitized_artist}-{sanitized_title}.jpg"
                        image_path = os.path.join(IMAGE_DIR, image_filename)

                        if not os.path.exists(image_path):
                            download_image(scraped_data['album_cover_url'], image_path)

                    # [개선] 결과 업데이트 또는 추가 로직
                    new_entry = {'artist': original_artist, 'title': original_title, **scraped_data}

                    # 기존에 있던 항목인지 확인 (가사만 있고 커버가 없던 경우)
                    existing_index = -1
                    for i, item in enumerate(all_results):
                        if item['artist'] == original_artist and item['title'] == original_title:
                            existing_index = i
                            break

                    if existing_index != -1:
                        all_results[existing_index].update(new_entry) # 기존 항목 업데이트
                    else:
                        all_results.append(new_entry) # 새 항목 추가
                else:
                    print(f"  ❌ 수집 실패. 실패 목록에 기록합니다.")
                    with open(FAILED_FILE, "a", encoding="utf-8") as f:
                        f.write(f"{original_artist}\t{original_title}\n")

            except Exception as e:
                print(f"  🚨 루프 내에서 예상치 못한 오류 발생: {e}")
                with open(FAILED_FILE, "a", encoding="utf-8") as f:
                    f.write(f"{original_artist}\t{original_title}\n")
            finally:
                if driver:
                    driver.quit()
                time.sleep(random.uniform(2, 4))

        # [개선] 모든 작업이 끝난 후, 최종 결과를 CSV 파일에 한 번에 저장
        print("\n3단계: 최종 수집 결과를 파일에 저장 중...")
        final_df = pd.DataFrame(all_results)
        # 컬럼 순서 정리
        final_df = final_df[['artist', 'title', 'source', 'lyrics', 'translation', 'album_cover_url', 'bugs_track_id']]
        final_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

        print("\n🎉 모든 가사 및 앨범 커버 수집/업데이트 작업이 종료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/

J-POP 가사/커버 수집 중:   0%|          | 0/460 [00:00<?, ?it/s]


🔍 수집 시도 (1/460): Kenshi Yonezu(켄시 요네즈/米津 玄師) - Lemon
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  🎶 벅스 가사 수집 성공!
  📸 앨범 커버 URL 확보!
  ✅ 이미지 저장 성공: 5151362_Kenshi Yonezu(켄시 요네즈米津 玄師)-Lemon.jpg

🔍 수집 시도 (2/460): NiziU (니쥬) - Make you happy
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  🎶 벅스 가사 수집 성공!
  📸 앨범 커버 URL 확보!
  ✅ 이미지 저장 성공: 31955846_NiziU (니쥬)-Make you happy.jpg

🔍 수집 시도 (3/460): Lisa Ono(리사 오노/小野リサ) - I Wish You Love
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  🎶 벅스 가사 수집 성공!
  📸 앨범 커버 URL 확보!
  ✅ 이미지 저장 성공: 288105_Lisa Ono(리사 오노小野リサ)-I Wish You Love.jpg

🔍 수집 시도 (4/460): RADWIMPS(래드윔프스) - なんでもないや (movie ver.) / Nandemonaiya (movie ver.)
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
